In [1]:
from lxml import etree
import pandas as pd
from uuid import uuid4

In [2]:
# Limit number of rows in dataframe display
pd.options.display.max_rows = 10

In [3]:
doc = etree.parse('list.xml')

### Build Taxon table

In [4]:
def getTaxonRank(scientificName):
    word_count = len(scientificName.split(' '))
    if word_count == 1:
        return('genus')
    else:
        return('species')

In [5]:
taxon_list = []
for section in doc.iterfind('section'):
    scientificName = section.findtext('crop/sci')
    taxonRank = getTaxonRank(scientificName)
    taxon_list.append({'kingdom':'Viridiplantae',
                       'scientificName': scientificName,
                       'taxonRank': taxonRank
                      })
    for pest in section.iterfind('pest'):
        scientificName = pest.findtext('sci')
        taxonRank = getTaxonRank(scientificName)
        taxon_list.append({'kingdom':'Animalia',
                           'scientificName': scientificName,
                           'taxonRank': taxonRank
                          })
df_taxon = pd.DataFrame(taxon_list)
df_taxon = df_taxon.drop_duplicates()

In [6]:
'''
kingdom: kingdom - col 0
scientificName: scientificName - col 1
taxonRank: taxonRank - col 2
ID: ID col 3
'''

if not ('ID' in df_taxon.columns):
    df_taxon['ID'] = None
    for index, row in df_taxon.iterrows():
        row['ID'] = uuid4().hex
else:
    print('ID column already exists')
df_taxon

,kingdom,scientificName,taxonRank,ID
0,Viridiplantae,Agave sisalana,species,baa848e144934b31bc1518c0f459952d
1,Animalia,Aonidiella orientalis,species,d446875b852b4f68927b2daf5273b724
2,Animalia,Dysmicoccus neobrevipes,species,607bf546c99a45d1afd51bc6117496ee
3,Viridiplantae,Persea americana,species,f131ce0370ab4c38b9bd3f54a83673da
4,Animalia,Trigonops,genus,c6276877a98a4b99b59a37e0a9846eb5
...,...,...,...,...
981,Animalia,Anua coronata,species,855cd310a1084e46aa8f3e602ffd4c27
983,Viridiplantae,Psophocarpus tetragonalubus,species,df9a93beff7f4328b4effe9a56ae0176
984,Animalia,Tetranychus neocaledonicus,species,a1179eea65ab4322b9335b2c7e857be9
986,Viridiplantae,Dioscorea,genus,444a65d4599841fcbcae962fb9316930


In [7]:
def get_ID(scientificName):
    ''' Returns the ID associated with a scintificName in the taxon table.'''
    x = df_taxon[df_taxon['scientificName'] == scientificName]['ID']
    return(x.values[0])

get_ID('Badamia exclamationis')

'7f2bfc135e5c4d058a80affdc5b6d2be'

### Build Distribution Table

Island records are encoded as follows:

List of insects and mites attacking crops in the Caroline (Belau-B, Yap-Y, Truk-T, Pohnpei-Po, Kosrae-K) and Mariana Islands (Guam-G, Saipan-S, Rota-R, and Commonwealth of Northern Marianas-C). New island records are indicated by n. Y indicates confirmation of a species previously reported as maybe present or needs verification. A ? indicates that the species may be there but its presence has not been verified with absolute certainty.

In [8]:
# This dictionary decodes island codes.

island_dict = {
    'M=?':[{'i':'Mariana Islands','n':False}],
    'M=C':[{'i':'Saipan','n':False},{'i':'Tinian','n':False},{'i':'Rota','n':False}],
    'M=G':[{'i':'Guam','n':False}],
    'M=RG':[{'i':'Guam','n':False},{'i':'Rota','n':False}],
    'M=S':[{'i':'Saipan','n':False}],
    'M=n':[{'i':'Saipan','n':True},{'i':'Tinian','n':True},{'i':'Rota','n':True},{'i':'Guam','n':True}],
    'M=x':[{'i':'Saipan','n':False},{'i':'Tinian','n':False},{'i':'Rota','n':False},{'i':'Guam','n':False}],
    'B=?':[{'i':'Republic of Palau','n':False}],
    'B=n':[{'i':'Republic of Palau','n':True}],
    'B=x':[{'i':'Republic of Palau','n':False}],
    'K=?':[{'i':'Kosrae State','n':False}],
    'K=n':[{'i':'Kosrae State','n':True}],
    'K=x':[{'i':'Kosrae State','n':False}], 
    'P=?':[{'i':'Pohnpei State','n':False}],
    'P=n':[{'i':'Pohnpei State','n':True}],
    'P=x':[{'i':'Pohnpei State','n':False}],
    'T=?':[{'i':'Chuuk State','n':False}],
    'T=Y':[{'i':'Chuuk State','n':False}],
    'T=n':[{'i':'Chuuk State','n':True}],
    'T=x':[{'i':'Chuuk State','n':False}],
    'Y=?':[{'i':'Yap State','n':False}],
    'Y=n':[{'i':'Yap State','n':True}],
    'Y=x':[{'i':'Yap State','n':False}],
} 

island_code = 'M=x'
mylist = island_dict[island_code]
for item in mylist: 
    print('{}: new_island_record={}'.format(item['i'], item['n']))

Saipan: new_island_record=False
Tinian: new_island_record=False
Rota: new_island_record=False
Guam: new_island_record=False


In [9]:
'''
ID: Core ID - col 0
island: locality - col 1
new_island_record: occurrenceRemarks - col 2
scientificName: skip - col 3
'''

dist_list = []
for section in doc.iterfind('section'):
    for pest in section.iterfind('pest'):
        scientificName = pest.findtext('sci')
        island_codes = pest.findall('island')
        for island_code in island_codes:
            island_list = island_dict[island_code.text]
            for item in island_list:
                if item['n']:
                    new_island_record = 'new island record'
                else:
                    new_island_record = ''
                dist_list.append({'scientificName': scientificName, 
                                  'island': item['i'],
                                  'new_island_record': new_island_record,
                                  'ID': get_ID(scientificName)
                             })
df_dist = pd.DataFrame(dist_list)
df_dist = df_dist.drop_duplicates()
df_dist

,ID,island,new_island_record,scientificName
0,d446875b852b4f68927b2daf5273b724,Yap State,,Aonidiella orientalis
1,d446875b852b4f68927b2daf5273b724,Saipan,,Aonidiella orientalis
2,d446875b852b4f68927b2daf5273b724,Tinian,,Aonidiella orientalis
3,d446875b852b4f68927b2daf5273b724,Rota,,Aonidiella orientalis
4,d446875b852b4f68927b2daf5273b724,Guam,,Aonidiella orientalis
...,...,...,...,...
4575,855cd310a1084e46aa8f3e602ffd4c27,Guam,,Anua coronata
4584,a1179eea65ab4322b9335b2c7e857be9,Guam,,Tetranychus neocaledonicus
4634,14ca520ccacc4507883c0d42b54db858,Republic of Palau,,Theretra nessus
4635,14ca520ccacc4507883c0d42b54db858,Yap State,,Theretra nessus


### Build Vernacular table

In [10]:
'''
ID: Core ID - col 0
scientificName: skip - col 1
vernacular: vernacularName - col 2
'''

vernacular_list = []
for section in doc.iterfind('section'):
    scientificName = section.findtext('crop/sci')
    common_names = section.findall('crop/com')
    for common_name in common_names:
        vernacular_list.append({'scientificName': scientificName, 
                                'vernacular': common_name.text,
                                'ID': get_ID(scientificName)
                               })    
        for pest in section.iterfind('pest'):
            scientificName = pest.findtext('sci')
            common_names = pest.findall('com')
            for common_name in common_names:
                vernacular_list.append({'scientificName': scientificName,
                                        'vernacular': common_name.text,
                                        'ID': get_ID(scientificName)
                                       })    
df_vernacular = pd.DataFrame(vernacular_list)
df_vernacular = df_vernacular.drop_duplicates()
df_vernacular

,ID,scientificName,vernacular
0,baa848e144934b31bc1518c0f459952d,Agave sisalana,Agave
1,d446875b852b4f68927b2daf5273b724,Aonidiella orientalis,oriental scale
2,607bf546c99a45d1afd51bc6117496ee,Dysmicoccus neobrevipes,grey pineapple mealybug
3,f131ce0370ab4c38b9bd3f54a83673da,Persea americana,Avocado
4,c6276877a98a4b99b59a37e0a9846eb5,Trigonops,weevil
...,...,...,...
1122,855cd310a1084e46aa8f3e602ffd4c27,Anua coronata,moth
1124,df9a93beff7f4328b4effe9a56ae0176,Psophocarpus tetragonalubus,Wing bean
1125,a1179eea65ab4322b9335b2c7e857be9,Tetranychus neocaledonicus,vegetable mite
1127,444a65d4599841fcbcae962fb9316930,Dioscorea,Yams


### Build ecological associates table

Note that this table has to be searched twice to discover all interactions for a taxon.

In [11]:
'''
sciNameID: Core ID - col 4
sciName: skip - col 3
associateSciName: scientificName - col 0
associateSciNameID: relatedResourceID - col 1
relationshipOfAssociate: relationsipOfResource - col 2
'''

associates_list = []
for section in doc.iterfind('section'):
    scientificName1 = section.findtext('crop/sci')
    for pest in section.iterfind('pest'):
        scientificName2 = pest.findtext('sci')
        associates_list.append({'sciNameID': get_ID(scientificName1),
                                'sciName': scientificName1, 
                                'associateSciNameID': get_ID(scientificName2),
                                'associateSciName': scientificName2,
                                'relationshipOfAssociate': 'herbivore of',
                               })
        associates_list.append({'sciNameID': get_ID(scientificName2),
                                'sciName': scientificName2, 
                                'associateSciNameID': get_ID(scientificName1),
                                'associateSciName': scientificName1,
                                'relationshipOfAssociate': 'host plant of'
                               })

df_associates = pd.DataFrame(associates_list)
df_associates = df_associates.drop_duplicates()
df_associates

,associateSciName,associateSciNameID,relationshipOfAssociate,sciName,sciNameID
0,Aonidiella orientalis,d446875b852b4f68927b2daf5273b724,herbivore of,Agave sisalana,baa848e144934b31bc1518c0f459952d
1,Agave sisalana,baa848e144934b31bc1518c0f459952d,host plant of,Aonidiella orientalis,d446875b852b4f68927b2daf5273b724
2,Dysmicoccus neobrevipes,607bf546c99a45d1afd51bc6117496ee,herbivore of,Agave sisalana,baa848e144934b31bc1518c0f459952d
3,Agave sisalana,baa848e144934b31bc1518c0f459952d,host plant of,Dysmicoccus neobrevipes,607bf546c99a45d1afd51bc6117496ee
4,Trigonops,c6276877a98a4b99b59a37e0a9846eb5,herbivore of,Persea americana,f131ce0370ab4c38b9bd3f54a83673da
...,...,...,...,...,...
1893,Dioscorea,444a65d4599841fcbcae962fb9316930,host plant of,Diaphania hyalinata,ead2d3e34b7d42108ee942a25e4e5080
1894,Adoxophyes melia,3b57d1c2347a4d07a9864782dd33de77,herbivore of,Dioscorea,444a65d4599841fcbcae962fb9316930
1895,Dioscorea,444a65d4599841fcbcae962fb9316930,host plant of,Adoxophyes melia,3b57d1c2347a4d07a9864782dd33de77
1896,Spodoptera litura,b8b8b53ecbcb4c33aa82888e609fee2b,herbivore of,Dioscorea,444a65d4599841fcbcae962fb9316930


In [12]:
import os

if not os.path.exists('dwca'):
    os.makedirs('dwca')
df_taxon.to_csv('dwca/taxon.csv', index=False)
df_dist.to_csv('dwca/distribution.csv', index=False)
df_vernacular.to_csv('dwca/vernacular.csv', index=False)
df_associates.to_csv('dwca/associates.csv', index=False)

In [13]:
# Hack to remove qouble quotes from vernacular.csv
#str = open('dwca/vernacular.csv', 'rt').read()
#str = str.replace('"', '')
#open("dwca/vernacular.csv", "wt").write(str) 